In [1]:
import os, sys
import requests
from PS_time_series import *

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate() # do NOT select read only scopes
    ee.Initialize()
    
help(PS_monthly_TS)

Help on function PS_monthly_TS in PS_time_series:

PS_monthly_TS(region_num, chip_shape, poly_shape, start_date, end_date, plot_YYYYMM, export_composite=False)
    region_num = chip region number
    chip_shape = full file path to user_train chip region shapefile 
    poly_shape = full file path to user_train poly region shapefile 
    start_date = planet monthly time series start date 
    end_date = planet monthly time series end date 
    plot_YYYYMM = display 3 dates(YYYY-MM) in a list as RGB 
    export_composite = folder to export planet monthly NDVI timeseries



In [ ]:

import os, sys, glob, time
import numpy as np
import pandas as pd
import io
import requests
import ee

from shapely.geometry import Polygon, mapping
import geopandas as gpd
import geemap
import geemap.geemap as geemap
import localtileserver 
import jupyter_contrib_nbextensions 
import ipyleaflet
import ipywidgets as widgets

#import rasterio as rio

def GET_GEE(gee_name, start, end, aoi_shape, region):
    gdf = gpd.read_file(aoi_shape)
    gdf.crs = "EPSG:8858"
    
    gdf_web = gdf.to_crs("EPSG:4326")
    gdf_web = gdf_web[gdf_web["region"] == str(region)]
    aoi = ee.Geometry.Rectangle([gdf_web.bounds.minx.min(), gdf_web.bounds.miny.min(), gdf_web.bounds.maxx.max(), gdf_web.bounds.maxy.max()])

    collection = ee.ImageCollection(gee_name).filterBounds(aoi).filterDate(start, end)
    clip_col = collection.map(lambda col : col.clip(aoi))

    return clip_col

def getNDVI(image):
    ndvi = image.normalizedDifference(['N','R']).rename("NDVI")
    #image = image.addBands(ndvi)
    return(ndvi)

def PS_monthly_TS(region_num, chip_shape, poly_shape, start_date, end_date, plot_YYYYMM, export_composite=False):
    '''
    region_num = chip region number
    chip_shape = full file path to user_train chip region shapefile 
    poly_shape = full file path to user_train poly region shapefile 
    start_date = planet monthly time series start date 
    end_date = planet monthly time series end date 
    plot_YYYYMM = display 3 dates(YYYY-MM) in a list as RGB 
    export_composite = folder to export planet monthly NDVI timeseries 
    '''
    start = time.time()

    poly_gdf = gpd.read_file(poly_shape)
    poly_gdf.crs = "EPSG:8858"
    web_polys = poly_gdf.to_crs("EPSG:4326")
    web_polys = web_polys[web_polys["region"] == str(region_num)]
    gdf = gpd.read_file(chip_shape)
    gdf.crs = "EPSG:8858"
    gdf_web = gdf.to_crs("EPSG:4326")
    gdf_web = gdf_web[gdf_web["region"] == str(region_num)]
    #for Polygon geo data type
    geoms = [i for i in web_polys.geometry]
    geomss = [i for i in gdf_web.geometry]
    features=[]
    for i in range(len(geoms)):
        g = [i for i in web_polys.geometry]
        x,y = g[i].exterior.coords.xy
        cords = np.dstack((x,y)).tolist()
        g=ee.Geometry.Polygon(cords)
        feature = ee.Feature(g)
        features.append(feature)
        ee_object = ee.FeatureCollection(features)
    featuress=[]
    for ii in range(len(geomss)):
        ge = [i for i in gdf_web.geometry]
        xx,yy = ge[ii].exterior.coords.xy
        cordss = np.dstack((xx,yy)).tolist()
        ge=ee.Geometry.Polygon(cordss)
        featuree = ee.Feature(ge)
        featuress.append(featuree)
        chip_ee_object = ee.FeatureCollection(featuress)
        
    planet_NDVI = GET_GEE(gee_name="projects/planet-nicfi/assets/basemaps/americas", 
                 start=start_date, end=end_date, 
                 aoi_shape=chip_shape, 
                 region=region_num).map(getNDVI).toBands()

    YYYYMM_bands = ['planet_medres_normalized_analytic_'+str(i)+'_mosaic_NDVI' for i in plot_YYYYMM]
    band_list = [i['id'] for i in planet_NDVI.getInfo()['bands']]
    plotNDVIbands = [i for i in band_list if i in YYYYMM_bands]
    planet_NDVI=ee.Image(planet_NDVI).rename(band_list)

    if not export_composite == False:
        if not os.path.exists(export_composite):
            os.makedirs(export_composite)
        out_file = os.path.join(export_composite, "PS_monthly_"+str(region_num)+".tif")

        planet_NDVI_export = planet_NDVI.resample('bilinear').reproject(crs='EPSG:8858', scale=4.7)
        geemap.ee_export_image(planet_NDVI, out_file, scale=4.7) 


    Map = geemap.Map(center=(float(gdf_web.geometry.centroid.y.values), float(gdf_web.geometry.centroid.x.values)), zoom=15)
    #Map.add_basemap('HYBRID')
    Map.add_tile_layer('http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}', name='Google Satellite', attribution='Google')
    Map.addLayer(planet_NDVI, {'bands':plotNDVIbands, 'min': 0.0, 'max': 1.0, 'gamma':[1,1,1]}, name="Planet monthly NDVI", shown=True, opacity=1.0)
    Map.addLayer(chip_ee_object.style(**{'color': '000000', 'fillColor': '00000000'}), {},"Training chips")
    Map.addLayer(ee_object.style(**{'color': 'maroon', 'fillColor': '00000000'}), {},"Training digitizations")
    
    end = time.time()
    print('this took '+str(float(end)-float(start))+' seconds')
    
    Map
    return Map


In [3]:
region_num = 394702
out_folder = "/planet_composites"
start_date = "2020-07-01"
end_date = "2021-08-01"
plot_YYYYMM = ["2020-09", "2020-12", "2021-03"]
chip_shape = "/training-digitizations/PyCropSeg_Chips21_P1P2_8858sq.shp"
poly_shape = "/training-digitizations/PyCropSeg_Polys21_P1P2_8858sq.shp"

PS_monthly_TS(region_num, chip_shape, poly_shape, start_date, end_date, plot_YYYYMM, export_composite=out_folder)

['planet_medres_normalized_analytic_2020-09_mosaic_NDVI', 'planet_medres_normalized_analytic_2020-12_mosaic_NDVI', 'planet_medres_normalized_analytic_2021-03_mosaic_NDVI']
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Lauren\Downloads\P1P2_training_merged\planet_composites\PS_monthly_394702.tif


C:\Users\Lauren\Downloads\P1P2_training_merged\scripts\PY_time_series.py:108: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Map = geemap.Map(center=(float(gdf_web.geometry.centroid.y.values), float(gdf_web.geometry.centroid.x.values)), zoom=15)


this took 4.720506191253662 seconds


Map(center=[-26.29267722125277, -55.651553268946415], controls=(WidgetControl(options=['position', 'transparen…